# Sweep Run Analysis

This notebook analyzes the results of an experiment run with `SweepRunner`.
It loads sweep-level and trial-level results from `data/<experiment_name>`, including sweep parameters and outcomes.

In [ ]:
import os
import json
import pandas as pd
from glob import glob
from collections import defaultdict
import numpy as np
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.insert(0, root_path)
import matplotlib.pyplot as plt
%matplotlib inline

experiment_name = "tutorial_sweep_runner"

# If you want the statistics with specific outcomes excluded, you can set this list.
exclude_outcomes = [] # ["exception"]

## Locate all sweep result folders

In [ ]:
experiment_dir = os.path.join("..", "data", experiment_name)
assert os.path.isdir(experiment_dir), f"Directory not found: {experiment_dir}"
sweep_dirs = sorted([d for d in glob(os.path.join(experiment_dir, "sweeps", "sweep_*")) if os.path.isdir(d)])
print(f"Found {len(sweep_dirs)} sweeps.")

## Load all sweep parameters and trial outcomes

In [ ]:
aggregated_stats_path = os.path.join(experiment_dir, "aggregated_stats.json")
if not os.path.exists(aggregated_stats_path):
    aggregated_stats_path = os.path.join(experiment_dir, "aggregated_stats_interim.json")
    if not os.path.exists(aggregated_stats_path):
        raise FileNotFoundError(f"Aggregated stats file not found: Neither {os.path.join(experiment_dir, "aggregated_stats.json")} nor {os.path.join(experiment_dir, "aggregated_stats_interim.json")}")
with open(aggregated_stats_path, "r") as f:
    aggregated_stats = json.load(f)
trial_kwargs = aggregated_stats["trial_kwargs"]

sweep_records = []
trial_records = []
n_trials_list = []
for sweep_dir in sweep_dirs:
    params_path = os.path.join(sweep_dir, "params.json")
    result_path = os.path.join(sweep_dir, "result.json")
    if not (os.path.exists(params_path) and os.path.exists(result_path)):
        continue
    with open(params_path, "r") as f:
        params = json.load(f)
    with open(result_path, "r") as f:
        result = json.load(f)
    # Sweep-level record
    sweep_record = {
        k: (str(v) if isinstance(v, (list, dict)) else v)
        for k, v in params.items()
    }
    sweep_record.update({
        "sweep_dir": sweep_dir,
        "stats_abs": result.get("stats_abs", {}),
        "stats_rel": result.get("stats_rel", {}),
        "duration_stats": result.get("duration_stats", {}),
        "timestamp": result.get("timestamp", None)
    })
    n_trials = len(result.get("trial_outcomes", []))
    sweep_record["n_trials"] = n_trials
    n_trials_list.append(n_trials)
    sweep_records.append(sweep_record)
    # Trial-level records
    for trial in result.get("trial_outcomes", []):
        rec = dict(params)
        rec.update(trial)
        rec["sweep_dir"] = sweep_dir
        trial_records.append(rec)

df_sweeps = pd.DataFrame(sweep_records)
df_trials = pd.DataFrame(trial_records)
if exclude_outcomes:
    df_trials = df_trials[~df_trials["outcome"].isin(exclude_outcomes)].copy()
print(f"Loaded {len(df_sweeps)} sweeps and {len(df_trials)} trials.")
non_param_cols = ["sweep_dir", "stats_abs", "stats_rel", "duration_stats", "timestamp", "n_trials"]
param_cols = [c for c in df_sweeps.columns if c not in non_param_cols]
trial_non_param_cols = {"seed", "outcome", "duration", "trial", "sweep_dir", "exception", "traceback"}
trial_param_cols = [c for c in df_trials.columns if c not in trial_non_param_cols]

outcome_df = pd.DataFrame(df_sweeps["stats_abs"].tolist()).fillna(0).astype(int)
# Remove excluded outcomes
if exclude_outcomes:
    exclude_outcomes = [outcome for outcome in exclude_outcomes if outcome in outcome_df.columns]
    if len(exclude_outcomes) > 0:
        print(f"We will exclude outcomes: {exclude_outcomes}")
        outcome_df = outcome_df.drop(columns=exclude_outcomes, errors='ignore')
# Compute relative frequencies based on remaining outcomes
n_trials_eff = outcome_df.sum(axis=1)
outcome_df = outcome_df.div(n_trials_eff, axis=0) * 100

# Check if all sweeps have the same number of trials
if len(set(n_trials_list)) > 1:
    print("⚠️ Warning: Not all sweeps have the same number of trials! (corrupted data?)")
else:
    print(f"All sweeps have {n_trials_list[0]} trials.")


## Overview of sweep parameters and outcomes

In [ ]:
# Show unique values for each sweep parameter
print("Unique parameter values across all sweeps:")
for col in param_cols:
    uniques = df_sweeps[col].unique()
    print(f"- {col}: {uniques}")

print(f"\nAppearing outcomes: {outcome_df.columns.tolist()}")

## Overview of Exceptions

In [ ]:
if "exception" in df_trials.columns:
    df_exceptions = df_trials.dropna(subset=["exception"]).copy()

    if not df_exceptions.empty:
        exception_counts = (
            df_exceptions["exception"]
            .value_counts()
            .reset_index()
            .rename(columns={"exception": "message"})
        )

        limit_exception_prints = 10
        print(f"=== First {limit_exception_prints} Exceptions ===\n")
        exception_count = 0
        for msg, count in zip(exception_counts["message"], exception_counts["count"]):
            short_msg = msg.replace("\n", " ")[:100] + ("..." if len(msg) > 100 else "")
            print(f"{count} × {short_msg}")
            exception_count += 1
            if exception_count >= limit_exception_prints:
                break
                # pass

        # Have a look at the traceback of specific exception type
        exception_filter = "Q_"  # paste part of the exception string here
        limit_traceback_prints = 1
        df_filtered = df_exceptions[df_exceptions["exception"].str.contains(exception_filter, regex=False)]

        if not df_filtered.empty and "traceback" in df_filtered.columns:
            print(f"\n=== First {limit_traceback_prints} Traceback(s) for exceptions matching: '{exception_filter}' ===")
            traceback_count = 0
            for _, row in df_filtered.iterrows():
                sweep_dir = row.get("sweep_dir", "?")
                trial = row.get("trial", "?")
                seed = row.get("seed", "?")
                tb = row["traceback"]
                print(f"\n--- Sweep Directory {sweep_dir} | Trial {trial} | Seed {seed} ---\n")
                print(tb)
                traceback_count += 1
                if traceback_count >= limit_traceback_prints:
                    break
        else:
            print(f"\n(No tracebacks found for exceptions matching: '{exception_filter}')")
    else:
        print("No exceptions found 🎉")
else:
    print("'exception' column not found in df_trials.")


## Aggregated outcome statistics per parameter

In [ ]:
# Convert stats to DataFrame for aggregation
df_sweeps_outcomes = pd.concat([df_sweeps[param_cols], outcome_df], axis=1)
df_sweeps_outcomes.head()

In [ ]:
# Show aggregated statistics
print("\n=== Aggregated Outcome Statistics Across Sweeps ===\n")
display(df_sweeps_outcomes.describe())

In [ ]:
# Show mean outcome (relative) for each parameter value
for param in param_cols:
    print(f"\n=== Mean relative outcomes grouped by {param} ===")
    display(df_sweeps_outcomes.groupby(param)[outcome_df.columns].mean().round(5).astype(str) + "%")

## Plot outcome distribution

In [ ]:
outcome_df.sum().plot(kind="bar")
plt.title("Total outcome counts across all sweeps and trials")
plt.show()

## Extract all trials of given outcomes for Reproducability

In [ ]:
outcome_to_trials = defaultdict(list)

for _, row in df_trials.iterrows():
    params = {k: row[k] for k in trial_param_cols}
    for k, v in params.items():
        if isinstance(v, str) and ((v.startswith("[") and v.endswith("]")) or (v.startswith("{") and v.endswith("}"))):
            try:
                params[k] = eval(v)
            except Exception:
                pass
    entry = {"sweep_params": params, "seed": row["seed"], "sweep_dir": row["sweep_dir"]}
    outcome_to_trials[row["outcome"]].append(entry)

# Example for the first appearing outcome
first_outcome = next(iter(outcome_to_trials))
print(f"Example for outcome '{first_outcome}':")
for entry in outcome_to_trials[first_outcome][:5]:  # only show the first 5
    print(entry)
print(f"Total: {len(outcome_to_trials[first_outcome])} trials with outcome '{first_outcome}'")